# Upload Measurements and run analysis

Now you will post your measurement data and analysis to the database via the DoData app.

**If you are running the tutorials in DoLab, the following instructions are not necessary and you can skip directly to the next cell.**

You will need to authenticate to the database with your username and password. To make this easy, you can create a file called `.env` in this folder and complete it with your organization's URL and authentication information as follows:

```bash
dodata_url=https://animal.doplaydo.com
dodata_user=demo
dodata_password=yours
dodata_db=animal.dodata.db.doplaydo.com
dodata_db_user=full_access
dodata_db_password=yours

```

If you haven't defined a `.env` file or saved your credentials to your environment variables, you will be prompted for your credentials now.

In [ ]:
import getpass
import platform
from pathlib import Path

import pandas as pd
import requests
from httpx import HTTPStatusError
from tqdm.auto import tqdm

import doplaydo.dodata as dd

username = getpass.getuser()

Let's now create a project. 

In normal circumstances, everyone will be sharing and contributing to a project. In this demo, however, we want to *keep your project separate* from other users for clarity, so we will append your username to the project id. This way you can also safely delete and recreate projects without creating issues for others. If you prefer though, you can change the `PROJECT_ID` to anything you like. Just be sure to update it in the subsequent notebooks of this tutorial as well.

In [ ]:
PROJECT_ID = f"rings-{username}-{platform.node()}"
MEASUREMENTS_PATH = Path("6d4c615ff105/")

If you've been running this demo multiple times, you might already have one with the same name. Let's delete it so you can start fresh.

In [ ]:
try:
    print(dd.project.delete(project_id=PROJECT_ID).text)
except HTTPStatusError:
    pass

## New project

You can create the project, upload the design manifest, and upload the wafer definitions through the Webapp as well as programmatically using this notebook

### Upload Project

You can create a new project and extract all cells & devices below for the `RidgeLoss` and `RibLoss`

The expressions are regex expressions. For intro and testing your regexes you can check out [regex101](https://regex101.com)

To only extract top cells set `max_hierarchy_lvl=-1` and `min_hierarchy_lvl=-1`

To disable extraction use a max_hierarchy_lvl < min_hierarchy_lvl

Whitelists take precedence over blacklists, so if you define both, it uses only the whitelist.

In [ ]:
cell_extraction = [
    dd.project.Extraction(
        cell_id="rings",
        cell_white_list=["^RingDouble"],
        min_hierarchy_lvl=0,
        max_hierarchy_lvl=0,
    )
]

dd.project.create(
    project_id=PROJECT_ID,
    eda_file="test_chip.gds",
    lyp_file="generic.lyp",
    cell_extractions=cell_extraction,
).text

### Upload Design Manifest

The design manifest is a CSV file that includes all the cell names, the cell settings, a list of analysis to trigger, and a list of settings for each analysis.

In [ ]:
dm = pd.read_csv("design_manifest.csv")
dm

As you don't have an analysis defined yet you need to drop the columns that allow you to run automated analysis on measurement upload.

In [ ]:
dm = dm.drop(columns=["analysis", "analysis_parameters"])
dm

In [ ]:
dm.to_csv("design_manifest_without_analysis.csv", index=False)

In [ ]:
dd.project.upload_design_manifest(
    project_id=PROJECT_ID, filepath="design_manifest_without_analysis.csv"
).text

In [ ]:
dd.project.download_design_manifest(
    project_id=PROJECT_ID, filepath="design_manifest_downloaded.csv"
)

### Upload Wafer Definitions

The wafer definition is a JSON file where you can define the wafer names and die names and location for each wafer.

In [ ]:
dd.project.upload_wafer_definitions(
    project_id=PROJECT_ID, filepath="wafer_definitions.json"
).text

## Upload data

Your Tester can output the data in JSON files. It does not need to be Python.

You can get all paths which have measurement data within the test path.

In [ ]:
data_files = list(MEASUREMENTS_PATH.glob("**/data.json"))
print(data_files[0].parts)

 You should define a plotting per measurement type in python. Your plots can evolve over time even for the same measurement type.

Required:

```yaml
- x_name (str): x-axis name
- y_name (str): y-axis name
- x_col (str): x-column to plot
- y_col (list[str]): y-column(s) to plot; can be multiple
```


Optional:
```yaml

- scatter (bool): whether to plot as scatter as opposed to line traces
- x_units (str): x-axis units
- y_units (str): y-axis units
- x_log_axis (bool): whether to plot the x-axis on log scale
- y_log_axis (bool): whether to plot the y-axis on log scale
- x_limits (list[int, int]): clip x-axis data using these limits as bounds (example: [10, 100])
- y_limits (list[int, int]): clip y-axis data using these limits as bounds (example: [20, 100])
- sort_by (dict[str, bool]): columns to sort data before plotting. Boolean specifies whether to sort each column in ascending order.
                             (example: {"wavelegths": True, "optical_power": False})
- grouping (dict[str, int]): columns to group data before plotting. Integer specifies decimal places to round each column.
                             Different series will be plotted for unique combinations of x column, y column(s), and rounded column values.
                             (example: {"port": 1, "attenuation": 2})

```

In [ ]:
spectrum_measurement_type = dd.api.device_data.PlottingKwargs(
    x_name="wavelength",
    y_name="output_power",
    x_col="wavelength",
    y_col=["output_power"],
)

### Upload measurements

You can now upload measurement data.

This is a bare bones example, in a production setting, you can also add validation, logging, and error handling to ensure a smooth operation.

Every measurement you upload will trigger all the analysis that you defined in the design manifest.

In [ ]:
NUMBER_OF_THREADS = 1 if "127" in dd.settings.dodata_url else dd.settings.n_threads
wafer_set = set()
die_set = set()
NUMBER_OF_THREADS

In [ ]:
if NUMBER_OF_THREADS == 1:
    for path in tqdm(data_files):
        wafer_id = path.parts[0]
        die_x, die_y = path.parts[1].split("_")

        r = dd.api.device_data.upload(
            file=path,
            project_id=PROJECT_ID,
            wafer_id=wafer_id,
            die_x=die_x,
            die_y=die_y,
            device_id=path.parts[2],
            data_type="measurement",
            plotting_kwargs=spectrum_measurement_type,
        )
        wafer_set.add(wafer_id)
        die_set.add(path.parts[2])
        r.raise_for_status()

In [ ]:
data_files = list(MEASUREMENTS_PATH.glob("**/data.json"))
project_ids = []
device_ids = []
die_ids = []
die_xs = []
die_ys = []
wafer_ids = []
plotting_kwargs = []
data_types = []

for path in data_files:
    device_id = path.parts[2]
    die_id = path.parts[1]
    die_x, die_y = die_id.split("_")
    wafer_id = path.parts[0]

    device_ids.append(device_id)
    die_ids.append(die_id)
    die_xs.append(die_x)
    die_ys.append(die_y)
    die_set.add(die_id)
    wafer_ids.append(wafer_id)
    plotting_kwargs.append(spectrum_measurement_type)
    project_ids.append(PROJECT_ID)
    data_types.append("measurement")
    wafer_set.add(wafer_id)

In [ ]:
if NUMBER_OF_THREADS > 1:
    dd.device_data.upload_multi(
        files=data_files,
        project_ids=project_ids,
        wafer_ids=wafer_ids,
        die_xs=die_xs,
        die_ys=die_ys,
        device_ids=device_ids,
        data_types=data_types,
        plotting_kwargs=plotting_kwargs,
        progress_bar=True,
    )

## Analysis

You can run analysis at 3 different levels. For example to extract:

1. Device: averaged power envelope over certain number of samples.
2. Die: fit the propagation loss as a function of length.
3. Wafer: Define the Upper and Lower Spec limits for Known Good Die (KGD)

![](https://i.imgur.com/ZwIWS08.png)




To upload custom analysis functions to the DoData server, follow these simplified guidelines:

- Input:
  - Begin with a unique identifier (device_data_id, die_id, wafer_id) as the first argument.
  - Add necessary keyword arguments for the analysis.

- Output: Dictionary
  - output: Return a simple, one-level dictionary. All values must be serializable. Avoid using numpy or pandas; convert to lists if needed.
  - summary_plot: Provide a summary plot, either as a matplotlib figure or io.BytesIO object.
  - attributes: Add a serializable dictionary of the analysis settings.
  - device_data_id/die_id/wafer_id: Include the used identifier (device_data_id, die_id, wafer_id).


### Device analysis

You can either trigger analysis automatically by defining it in the design manifest, using the UI or using the Python DoData library.


In [ ]:
from IPython.display import Code, Image, display

import doplaydo.dodata as dd

display(Code(dd.config.Path.analysis_functions_device_fsr))

You can easily get a device pkey to try your device analsys:

In [ ]:
with dd.get_session() as session:
    device_data = dd.get_data_objects_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "radius_um", 20),
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        limit=1,
        session=session,
    )[0]
    device_data_pkey = device_data.pkey
    device_data_device_id = device_data.device.device_id
device_data_pkey

In [ ]:
response = dd.api.analysis_functions.validate(
    analysis_function_id="device_fsr",
    function_path=dd.config.Path.analysis_functions_device_fsr,
    test_model_pkey=device_data_pkey,
    target_model_name="device_data",
    parameters={"height": -0.02},
)
Image(response.content)

In [ ]:
response = dd.api.analysis_functions.validate_and_upload(
    analysis_function_id="device_fsr",
    function_path=dd.config.Path.analysis_functions_device_fsr,
    test_model_pkey=device_data.pkey,
    target_model_name="device_data",
    parameters={"height": -0.02},
)

### Trigger all analyses for all device data

You have 64 dies and you will get 64 analysis, one for each device.

In [ ]:
with dd.get_session() as session:
    device_data_objects = dd.get_data_objects_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        session=session,
    )
    device_data_pkeys = [d.pkey for d in device_data_objects]
    params = [{"height": -0.02}] * len(device_data_objects)
    analyses = dd.api.analysis.trigger_device_data_multi(
        device_data_pkeys=device_data_pkeys,
        analysis_function_id="device_fsr",
        parameters=params,
        progress_bar=True,
    )
    test_die_pkey = device_data_objects[0].die.pkey
    print(len(analyses))

### Die analysis

You can aggregate any metric for a die analysis, for example, we had 3 different ring radius. Each of which will have a different FSR.

In [ ]:
device_attributes = {"radius_um": 20, "gap_um": 0.2}

response = dd.api.analysis_functions.validate(
    analysis_function_id="die_aggregate",
    function_path=dd.config.Path.analysis_functions_die_aggregate,
    test_model_pkey=test_die_pkey,
    target_model_name="die",
    parameters={
        "device_attributes": device_attributes,
        "device_analysis_function": "device_fsr",
        "metric": "fsr_mean",
    },
)
Image(response.content)

In [ ]:
device_data_objects[0].die.pkey

In [ ]:
with dd.get_session() as session:
    device_data_objects = dd.get_data_objects_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        session=session,
    )
    device_id = device_data_objects[0].device.device_id
    response = dd.api.analysis_functions.validate_and_upload(
        analysis_function_id="die_aggregate",
        function_path=dd.config.Path.analysis_functions_die_aggregate,
        test_model_pkey=device_data_objects[0].die.pkey,
        target_model_name="die",
        parameters={
            "device_attributes": device_attributes,
            "device_analysis_function": "device_fsr",
            "metric": "fsr_mean",
        },
    )
    wafer_pkey = device_data_objects[0].die.wafer.pkey
    wafer_id = device_data_objects[0].die.wafer.wafer_id
response

### Wafer Analysis

Lets Define the Upper and Lower Spec limits for Known Good Die (KGD).

Lets find a wafer pkey for this project, so that we can trigger the die analysis on it.

In [ ]:
wafer_id

In [ ]:
parameters = {
    "device_id": device_id,
    "lower_spec": 4.60,
    "upper_spec": 4.75,
    "analysis_function_id": "device_fsr",
    "metric": "fsr_mean",
}

response = dd.api.analysis_functions.validate(
    analysis_function_id="wafer_aggregate",
    function_path=dd.config.Path.analysis_functions_wafer_device_data_id,
    test_model_pkey=wafer_pkey,
    target_model_name="wafer",
    parameters=parameters,
)
Image(response.content)

In [ ]:
response = dd.api.analysis_functions.validate_and_upload(
    analysis_function_id="wafer_aggregate",
    function_path=dd.config.Path.analysis_functions_wafer_device_data_id,
    test_model_pkey=wafer_pkey,
    target_model_name="wafer",
    parameters=parameters,
)

Now that you have uploaded the wafer analysis function you can trigger the wafer analysis for all the wafers so you can store and visualize the wafermaps in the DoData website.


In [ ]:
with dd.get_session() as session:
    device_data, df = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "radius_um", 20),
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        limit=1,
        session=session,
    )[0]
    print(device_data.device.device_id)

In [ ]:
with dd.get_session() as session:
    device_data20, df = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "radius_um", 20),
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        limit=1,
        session=session,
    )[0]
    device_data10, df = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "radius_um", 10),
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        limit=1,
        session=session,
    )[0]
    device_data5, df = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "radius_um", 5),
            dd.attribute_filter(dd.Cell, "gap_um", 0.2),
        ],
        limit=1,
        session=session,
    )[0]
    dd20_device_id = device_data20.device.device_id
    dd10_device_id = device_data10.device.device_id
    dd5_device_id = device_data5.device.device_id
    dd20_wafer_id = device_data20.die.wafer.wafer_id
    dd10_wafer_id = device_data10.die.wafer.wafer_id
    dd5_wafer_id = device_data5.die.wafer.wafer_id

You can trigger the device analysis for rings with 200nm gaps and 20, 10 and 5um.

64 dies and 3 devices will trigger 192 device analysis.

In [ ]:
with dd.get_session() as session:
    device_data_objects = dd.get_data_objects_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.or_(
                dd.Device.device_id == dd20_device_id,
                dd.Device.device_id == dd10_device_id,
                dd.Device.device_id == dd5_device_id,
            ),
        ],
        limit=1,
        session=session,
    )
    device_data_pkeys = [d.pkey for d in device_data_objects]
    params = [{"height": -0.02}] * len(device_data_objects)
    analyses = dd.api.analysis.trigger_device_data_multi(
        device_data_pkeys=device_data_pkeys,
        analysis_function_id="device_fsr",
        parameters=params,
        progress_bar=True,
    )
    print(len(analyses))

In [ ]:
wafer_set = {dd5_wafer_id}

In [ ]:
# 5um radius rings
for _wafer in tqdm(wafer_set):
    r = dd.analysis.trigger_wafer(
        project_id=PROJECT_ID,
        wafer_id=dd5_wafer_id,
        analysis_function_id="wafer_aggregate",
        parameters={
            "device_id": dd5_device_id,
            "lower_spec": 18.5,
            "upper_spec": 18.7,
            "analysis_function_id": "device_fsr",
            "metric": "fsr_mean",
        },
    )
    if r.status_code != 200:
        raise requests.HTTPError(r.text)

In [ ]:
# 10um radius rings
for _wafer in tqdm(wafer_set):
    r = dd.analysis.trigger_wafer(
        project_id=PROJECT_ID,
        wafer_id=dd10_wafer_id,
        analysis_function_id="wafer_aggregate",
        parameters={
            "device_id": dd10_device_id,
            "lower_spec": 9.30,
            "upper_spec": 9.40,
            "analysis_function_id": "device_fsr",
            "metric": "fsr_mean",
        },
    )
    if r.status_code != 200:
        raise requests.HTTPError(r.text)

In [ ]:
# 20um radius rings
for _wafer in tqdm(wafer_set):
    r = dd.analysis.trigger_wafer(
        project_id=PROJECT_ID,
        wafer_id=dd20_wafer_id,
        analysis_function_id="wafer_aggregate",
        parameters={
            "device_id": dd20_device_id,
            "lower_spec": 4.6,
            "upper_spec": 4.8,
            "analysis_function_id": "device_fsr",
            "metric": "fsr_mean",
        },
    )
    if r.status_code != 200:
        raise requests.HTTPError(r.text)

In [ ]:
plots = dd.analysis.get_wafer_analysis_plots(
    project_id=PROJECT_ID,
    wafer_id=wafer_id,
    target_model="wafer",
)
len(plots)

In [ ]:
for plot in plots:
    display(plot)